Uma loja de carro esportivo um portfolio de carros importados. E para melhor direcionar suas campanhas de venda e marketing eles precisam categorizar os carros pelo pais de cada montadora, dessa forma ela pode clusterizar seus carros pelo seu pais de origem.

- Carros fabricados em 2020 para frente
- Carros acima de USD 100,000.00

### Extract: Bucket S3

### Transformation: Aplicar Regras de Negocio

### Loading: Postgres Database

In [1]:
import boto3

s3 = boto3.client('s3')
response = s3.list_buckets()

# Output the bucket names
print('Existing buckets:')
for bucket in response['Buckets']:
    print(f'  {bucket["Name"]}')

Existing buckets:
  aws-glue-assets-764386922584-us-east-1
  bucket-testing-dnc-lucas-2024
  dnc-class-datasource-s3
  dnc-lucas-tutorial-class


In [2]:
# Listar as pastas de um bucket

# Listing folder from a bucket

bucket_name = 'dnc-lucas-tutorial-class'

prefix = ''
delimiter = '/'

response = s3.list_objects_v2(
    Bucket=bucket_name,
    Prefix=prefix,
    Delimiter=delimiter
)

folders = [prefix['Prefix'] for prefix in response.get('CommonPrefixes', [])]

folders

['Unsaved/',
 'cars/',
 'customers/',
 'ecommerce/',
 'loading/',
 'raw/',
 'redshift/',
 'uploading-from-api/',
 'uploading/',
 'worldcup/']

In [3]:
# Dentro da pasta raw, vamos ver quais objetos estao disponiveis

import boto3

# Initialize the S3 client
s3 = boto3.client('s3')

# Bucket name
bucket_name = 'dnc-lucas-tutorial-class'

# List all objects in the bucket
response = s3.list_objects_v2(
    Bucket=bucket_name,
    Prefix="raw/"
)

# Extract the list of objects
objects = response.get('Contents', [])

# Print the list of object keys (file names)
for obj in objects:
    print(obj['Key'])

raw/
raw/WorldCups.csv
raw/cars/
raw/cars/Sport car price.csv
raw/cars/car-companies-by-country-2024.csv
raw/ecommerce/olist_customers_dataset.csv
raw/ecommerce/olist_geolocation_dataset.csv
raw/ecommerce/olist_order_items_dataset.csv
raw/ecommerce/olist_order_payments_dataset.csv
raw/ecommerce/olist_order_reviews_dataset.csv
raw/ecommerce/olist_orders_dataset.csv
raw/ecommerce/olist_products_dataset.csv
raw/ecommerce/olist_sellers_dataset.csv
raw/ecommerce/product_category_name_translation.csv


In [4]:
import pandas as pd
from io import StringIO

# Bucket name
bucket_name = 'dnc-lucas-tutorial-class'
file_key = "raw/cars/Sport car price.csv"

response = s3.get_object(Bucket=bucket_name, Key=file_key)
csv_content = response['Body'].read().decode('utf-8')

df_portfolio = pd.read_csv(StringIO(csv_content))

df_portfolio.sample(10)

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
899,Porsche,Cayman,2022,2,300,280,4.5,"59,900"
69,Lamborghini,Aventador,2021,6.5,770,531,2.9,"417,826"
820,Jaguar,F-Type,2022,2,296,295,5.4,"62,500"
654,Nissan,370Z Nismo,2021,3.7,350,276,4.8,"45,000"
549,BMW,M2 CS,2022,3,444,406,3.8,"84,595"
40,Audi,RS3,2022,2.5,394,369,3.9,"57,000"
977,Mercedes-Benz,AMG GT,2021,4,523,494,3.8,"118,500"
212,Porsche,Panamera Turbo S,2021,4,620,604,2.9,"177,700"
723,Mercedes-Benz,SLS AMG,2021,4,730,590,2.9,"250,000"
173,TVR,Griffith,2022,5,500,479,3.8,"123,500"


In [5]:
df_portfolio["Year"].value_counts()

Year
2021    576
2022    382
2020     25
2015     12
2019      3
2017      3
2023      3
2014      2
1965      1
Name: count, dtype: int64

In [6]:
df_portfolio["Price (in USD)"].value_counts().sort_values()

Price (in USD)
3,900,000     1
49,000        1
222,500       1
67,800        1
78,450        1
             ..
114,000      16
58,900       17
625,000      22
3,000,000    24
500,000      34
Name: count, Length: 367, dtype: int64

In [7]:
df_portfolio["Car Make"].value_counts()

Car Make
Porsche          88
McLaren          75
Audi             71
Lamborghini      66
BMW              63
Chevrolet        60
Ferrari          55
Mercedes-Benz    54
Aston Martin     50
Ford             48
Dodge            41
Nissan           37
Lotus            35
Jaguar           30
Lexus            26
Maserati         25
Bentley          25
Bugatti          23
Tesla            19
Alfa Romeo       16
Acura            16
Koenigsegg       15
Rimac            14
Pagani           12
Mercedes-AMG     11
Rolls-Royce      10
Toyota            5
W Motors          3
Subaru            3
Pininfarina       2
TVR               2
Kia               1
Alpine            1
Ariel             1
Shelby            1
Mazda             1
Polestar          1
Ultima            1
Name: count, dtype: int64

In [8]:
import pandas as pd
from io import StringIO

# Bucket name
bucket_name = 'dnc-lucas-tutorial-class'
file_key = "raw/cars/car-companies-by-country-2024.csv"

response = s3.get_object(Bucket=bucket_name, Key=file_key)
csv_content = response['Body'].read().decode('utf-8')

df_country = pd.read_csv(StringIO(csv_content))

df_country.sample(10)

,country,CarCompaniesNumOfCarCompanies,CarCompaniesMostPopularBrands
55,Liechtenstein,1,NanoFlowcell
17,Italy,61,"Abarth, Alfa Romeo, Aprilia, Ferrari, Fiat, La..."
54,Andorra,1,TransAm Depot
56,Monaco,1,Monte Carlo
21,Poland,8,"Leopard, Stoewer"
3,Pakistan,1,Adam
12,Germany,114,"Audi, BMW, Mercedes-Benz, Opel, Porsche, Smart..."
31,Netherlands,9,Lightyear
8,Japan,23,"Mazda, Mitsubishi, Nissan, Subaru, Suzuki, Toy..."
1,China,124,"Chery Automobile, Changan Automobile, CFMoto, ..."


In [9]:
df_country.iloc[1]

country                                                                      China
CarCompaniesNumOfCarCompanies                                                  124
CarCompaniesMostPopularBrands    Chery Automobile, Changan Automobile, CFMoto, ...
Name: 1, dtype: object

In [10]:
df_country['CarCompaniesMostPopularBrands'] = df_country["CarCompaniesMostPopularBrands"].str.split(',')
df_country

,country,CarCompaniesNumOfCarCompanies,CarCompaniesMostPopularBrands
0,India,7,"[Royal Enfield, Tata Motors, TVS, Force Mot..."
1,China,124,"[Chery Automobile, Changan Automobile, CFMot..."
2,United States,231,"[Buick, Cadillac, Chevrolet, Chrysler, Dod..."
3,Pakistan,1,[Adam]
4,Nigeria,1,[Innoson]
5,Brazil,10,"[MP Lafer, Troller]"
6,Russia,24,"[AvtoVAZ, Lada, UAZ, Moskvich]"
7,Mexico,3,"[Inferno, Mastretta, VUHL]"
8,Japan,23,"[Mazda, Mitsubishi, Nissan, Subaru, Suzuki..."
9,Vietnam,1,[VinFast]


In [13]:
df_country.iloc[0]

country                                  India
CarCompaniesNumOfCarCompanies                7
Car Make                         Royal Enfield
Name: 0, dtype: object

In [12]:
df_country = df_country.explode('CarCompaniesMostPopularBrands').reset_index(drop=True)
df_country.rename(columns={"CarCompaniesMostPopularBrands": "Car Make"}, inplace=True)
df_country

,country,CarCompaniesNumOfCarCompanies,Car Make
0,India,7,Royal Enfield
1,India,7,Tata Motors
2,India,7,TVS
3,India,7,Force Motors
4,China,124,Chery Automobile
...,...,...,...
146,Estonia,2,Nobe
147,Estonia,2,TARK
148,Andorra,1,TransAm Depot
149,Liechtenstein,1,NanoFlowcell


In [14]:
df_country[df_country["country"]=="China"]

,country,CarCompaniesNumOfCarCompanies,Car Make
4,China,124,Chery Automobile
5,China,124,Changan Automobile
6,China,124,CFMoto
7,China,124,BYD Auto
8,China,124,Great Wall
9,China,124,Haval
10,China,124,LDV
11,China,124,Baojun
12,China,124,Beijing Automobile Works
13,China,124,BJEV


In [15]:
df_portfolio.sample(2)

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD)
429,Mercedes-Benz,AMG C 63,2022,4,503,516,3.8,"70,000"
398,Maserati,GranTurismo,2022,4.7,454,384,4.7,"134,300"


In [19]:
df_portfolio["key"] = df_portfolio["Car Make"].str.lower().str.replace(' ', '').str.replace('[^a-zA-Z0-9]', '')
df_country["key"] = df_country["Car Make"].str.lower().str.replace(' ', '').str.replace('[^a-zA-Z0-9]', '')

In [20]:
df_portfolio.sample(2)

,Car Make,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD),key
236,Alfa Romeo,4C,2020,1.7,237,258,4.1,"67,150",alfaromeo
50,Mercedes-Benz,AMG C63,2021,4,503,516,3.8,"68,000",mercedes-benz


In [22]:
df_country.sample(2)

,country,CarCompaniesNumOfCarCompanies,Car Make,key
141,Lithuania,2,Forveda,forveda
132,Denmark,2,Zenvo,zenvo


In [23]:
df_merged = pd.merge(df_portfolio, df_country, on="key", how="left")
df_merged

,Car Make_x,Car Model,Year,Engine Size (L),Horsepower,Torque (lb-ft),0-60 MPH Time (seconds),Price (in USD),key,country,CarCompaniesNumOfCarCompanies,Car Make_y
0,Porsche,911,2022,3,379,331,4,"101,200",porsche,Germany,114.0,Porsche
1,Lamborghini,Huracan,2021,5.2,630,443,2.8,"274,390",lamborghini,Italy,61.0,Lamborghini
2,Ferrari,488 GTB,2022,3.9,661,561,3,"333,750",ferrari,Italy,61.0,Ferrari
3,Audi,R8,2022,5.2,562,406,3.2,"142,700",audi,Germany,114.0,Audi
4,McLaren,720S,2021,4,710,568,2.7,"298,000",mclaren,United Kingdom,157.0,McLaren
...,...,...,...,...,...,...,...,...,...,...,...,...
1002,Koenigsegg,Jesko,2022,5,1280,1106,2.5,"3,000,000",koenigsegg,NaN,NaN,NaN
1003,Lotus,Evija,2021,Electric Motor,1972,1254,2,"2,000,000",lotus,United Kingdom,157.0,Lotus
1004,McLaren,Senna,2021,4,789,590,2.7,"1,000,000",mclaren,United Kingdom,157.0,McLaren
1005,Pagani,Huayra,2021,6,764,738,3,"2,600,000",pagani,NaN,NaN,NaN


In [24]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1007 entries, 0 to 1006
Data columns (total 12 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Car Make_x                     1007 non-null   object 
 1   Car Model                      1007 non-null   object 
 2   Year                           1007 non-null   int64  
 3   Engine Size (L)                997 non-null    object 
 4   Horsepower                     1007 non-null   object 
 5   Torque (lb-ft)                 1004 non-null   object 
 6   0-60 MPH Time (seconds)        1007 non-null   object 
 7   Price (in USD)                 1007 non-null   object 
 8   key                            1007 non-null   object 
 9   country                        935 non-null    object 
 10  CarCompaniesNumOfCarCompanies  935 non-null    float64
 11  Car Make_y                     935 non-null    object 
dtypes: float64(1), int64(1), object(10)
memory usage

In [25]:
df_merged["country"].value_counts()

country
Germany                 276
United Kingdom          215
United States           168
Italy                   162
Japan                    72
France                   23
Croatia                  14
United Arab Emirates      3
Sweden                    1
South Korea               1
Name: count, dtype: int64

In [26]:
df_merged = df_merged[['Car Make_x',
                        'Car Model',
                        'Year',
                        'Price (in USD)',
                        'country']]

In [27]:
df_merged

,Car Make_x,Car Model,Year,Price (in USD),country
0,Porsche,911,2022,"101,200",Germany
1,Lamborghini,Huracan,2021,"274,390",Italy
2,Ferrari,488 GTB,2022,"333,750",Italy
3,Audi,R8,2022,"142,700",Germany
4,McLaren,720S,2021,"298,000",United Kingdom
...,...,...,...,...,...
1002,Koenigsegg,Jesko,2022,"3,000,000",NaN
1003,Lotus,Evija,2021,"2,000,000",United Kingdom
1004,McLaren,Senna,2021,"1,000,000",United Kingdom
1005,Pagani,Huayra,2021,"2,600,000",NaN


In [28]:
df_merged["Price (in USD)"] = df_merged["Price (in USD)"].str.replace(",", "").astype(float)

/tmp/ipykernel_34734/4057544339.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged["Price (in USD)"] = df_merged["Price (in USD)"].str.replace(",", "").astype(float)


In [29]:
df_merged

,Car Make_x,Car Model,Year,Price (in USD),country
0,Porsche,911,2022,101200.0,Germany
1,Lamborghini,Huracan,2021,274390.0,Italy
2,Ferrari,488 GTB,2022,333750.0,Italy
3,Audi,R8,2022,142700.0,Germany
4,McLaren,720S,2021,298000.0,United Kingdom
...,...,...,...,...,...
1002,Koenigsegg,Jesko,2022,3000000.0,NaN
1003,Lotus,Evija,2021,2000000.0,United Kingdom
1004,McLaren,Senna,2021,1000000.0,United Kingdom
1005,Pagani,Huayra,2021,2600000.0,NaN


In [30]:
df_merged_filtered = df_merged[df_merged['Year'] >= 2020]
df_merged_filtered = df_merged_filtered[df_merged_filtered['Price (in USD)'] >= 100000]
df_merged_filtered = df_merged_filtered.dropna(subset=["country"], axis=0)

In [31]:
df_merged_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 555 entries, 0 to 1006
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Car Make_x      555 non-null    object 
 1   Car Model       555 non-null    object 
 2   Year            555 non-null    int64  
 3   Price (in USD)  555 non-null    float64
 4   country         555 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 26.0+ KB


In [32]:
df_merged_filtered.sample(10)

,Car Make_x,Car Model,Year,Price (in USD),country
497,Aston Martin,DBS Superleggera,2022,314500.0,United Kingdom
863,Bentley,Continental GT,2021,202500.0,United Kingdom
454,Ferrari,F8 Spider,2021,280000.0,Italy
874,Ford,GT,2022,500000.0,United States
994,McLaren,GT,2022,210000.0,United Kingdom
583,Ford,GT,2022,500000.0,United States
691,Audi,RS 6 Avant,2021,110000.0,Germany
366,Audi,RS7,2021,114000.0,Germany
315,Maserati,GranTurismo,2022,134300.0,Italy
466,Ford,GT,2022,500000.0,United States


In [33]:
# Se voce nao tiver um banco postgres

# docker run --name dnc-datasources -e POSTGRES_PASSWORD=dnc123 -e POSTGRES_USER:dnc -e POSTGRES_DB:datasource -p 5432:5432 -d postgres

In [34]:
from sqlalchemy import create_engine
import pandas as pd

username = 'postgres'
password = 'dnc123'
host = 'localhost'
port = 5432
database_name = 'postgres'

engine = create_engine(f'postgresql://{username}:{password}@{host}:{port}/{database_name}')

In [35]:
df_merged_filtered.to_sql('cars_by_countries_to_sales_campaign', con=engine, if_exists='replace', index=False)

555